## Install Packages

In [1]:
!conda install -y -c conda-forge faiss-gpu
!apt-get -y update
!apt-get -y install libatlas-base-dev

done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.14.0
  latest version: 25.7.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.

Retrieving notices: ...working... done
Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease                      
Hit:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease              
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease       
Hit:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease       
Reading package lists... Done                     
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libatlas-base-dev is already the newest version (3.10.3-5).
0 upgraded, 0 newly installed, 0 to remove and 83 not upgraded.


## Load Data

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.random_projection import GaussianRandomProjection

from tqdm import tqdm

import faiss

In [3]:
df = pd.read_csv("story_dataset.csv")
df

,prompt_id,prompt,story,hidden_state_file,len_generated_story,len_new_story
0,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Blaz...,./hidden_states/prompt_1.npz,270,271
1,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Spar...,./hidden_states/prompt_1.npz,349,350
2,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Scor...,./hidden_states/prompt_1.npz,278,278
3,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,117,118
4,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,129,130
...,...,...,...,...,...,...
9995,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,289,290
9996,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,119,119
9997,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,127,128
9998,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,441,441


In [4]:
max_story_len = max(df["len_generated_story"])
max_story_len

522

In [5]:
hidden_states_by_layer = {}
NUM_PROMPTS = 10

for prompt_id in range(1, NUM_PROMPTS + 1):
    with np.load(f'./hidden_states/prompt_{prompt_id}.npz') as loaded_data:
        for i in tqdm(range(1000)):
            curr_hidden_states = loaded_data[f"arr_{i}"][0]
#             print(curr_hidden_states.shape)
            for layer in range(1):
                padded_arr = np.zeros((max_story_len, 512))
                padded_arr_len = len(curr_hidden_states[layer][0])
                
                padded_arr[:padded_arr_len] = curr_hidden_states[layer][0]
                
                padded_arr = padded_arr.flatten().astype('float32') #FAISS expects data in type float32 instead of float64 - saves memory too!
#                 print(padded_arr.shape)
                
                if(f"layer_{layer}" in hidden_states_by_layer):
                    hidden_states_by_layer[f"layer_{layer}"].append(padded_arr)
                else:
                    hidden_states_by_layer[f"layer_{layer}"] = [padded_arr]

100%|███████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:39<00:00, 10.05it/s]


In [6]:
layer_hs_array = np.array(hidden_states_by_layer["layer_0"])
layer_hs_array.shape

(10000, 267264)

## Layer 0 Clustering

In [7]:
# Use original vectors for clustering - uncomment next line and comment out last two lines

dim_reduced_vecs = layer_hs_array

# random_projector = GaussianRandomProjection(random_state = 42)
# dim_reduced_vecs = random_projector.fit_transform(layer_hs_array).astype('float32')

In [8]:
dim_reduced_vecs = np.array([v / np.linalg.norm(v) for v in dim_reduced_vecs])
dim_reduced_vecs.shape

(10000, 267264)

In [9]:
# K-means Clustering

ncentroids = NUM_PROMPTS
niter = 20
verbose = True
dim = dim_reduced_vecs.shape[1]
kmeans = faiss.Kmeans(dim, ncentroids, niter = niter, verbose = verbose, gpu = True, nredo = 10, spherical = True, max_points_per_centroid = 1000)
kmeans.train(dim_reduced_vecs)

Clustering 10000 points in 267264D to 10 clusters, redo 10 times, 20 iterations
  Preprocessing in 3.25 s
Outer iteration 0 / 10
  Iteration 19 (55.09 s, search 36.54 s): objective=2965.37 imbalance=1.207 nsplit=0       
Objective improved: keep new clusters
Outer iteration 1 / 10
  Iteration 19 (111.38 s, search 74.20 s): objective=2989.07 imbalance=1.347 nsplit=0       
Objective improved: keep new clusters
Outer iteration 2 / 10
  Iteration 19 (165.98 s, search 110.92 s): objective=2945.85 imbalance=1.556 nsplit=0       
Outer iteration 3 / 10
  Iteration 19 (222.57 s, search 148.72 s): objective=2975.02 imbalance=1.214 nsplit=0       
Outer iteration 4 / 10
  Iteration 19 (276.64 s, search 184.98 s): objective=2929.77 imbalance=1.363 nsplit=0       
Outer iteration 5 / 10
  Iteration 19 (333.27 s, search 222.38 s): objective=2987.59 imbalance=1.169 nsplit=0       
Outer iteration 6 / 10
  Iteration 19 (388.30 s, search 259.17 s): objective=2956.03 imbalance=1.115 nsplit=0       
Ou

2989.072021484375

In [10]:
kmeans.centroids #cluster centers

array([[-8.49578716e-03, -5.35234623e-03,  1.66376019e-04, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-6.32371288e-03, -3.98394046e-03,  1.23839360e-04, ...,
        -5.04619777e-07, -3.79645871e-06,  3.50442338e-06],
       [-8.90896935e-03, -5.61264716e-03,  1.74467175e-04, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [-7.57020479e-03, -4.76923026e-03,  1.48249834e-04, ...,
         1.11156169e-05, -2.62718277e-05, -9.82280744e-06],
       [-9.27523803e-03, -5.84339630e-03,  1.81640004e-04, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-6.06259378e-03, -3.81943560e-03,  1.18725766e-04, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]], dtype=float32)

In [11]:
for centroid in kmeans.centroids:
    print(np.linalg.norm(centroid))

1.0000052
1.0000137
1.0000044
1.0000288
1.0000038
1.0000069
1.000015
1.000034
1.0000038
1.0000228


In [12]:
kmeans.obj #inertia at each iteration

array([ 991.94604492, 2805.61914062, 2925.296875  , 2949.68701172,
       2956.21899414, 2960.59155273, 2962.2800293 , 2963.14135742,
       2963.78417969, 2964.26879883, 2964.62915039, 2964.90551758,
       2965.11474609, 2965.27392578, 2965.34008789, 2965.36376953,
       2965.37207031, 2965.37207031, 2965.37207031, 2965.37207031,
       1024.02978516, 2823.59838867, 2905.96118164, 2940.24389648,
       2965.79174805, 2981.31152344, 2984.4855957 , 2985.8112793 ,
       2986.20996094, 2986.48461914, 2986.63842773, 2986.80615234,
       2987.07519531, 2987.6652832 , 2988.2878418 , 2988.5144043 ,
       2988.64331055, 2988.74365234, 2988.88964844, 2989.07202148])

In [13]:
normalized_vecs = [v / np.linalg.norm(v) for v in dim_reduced_vecs]

In [14]:
cos_similarities = normalized_vecs @ kmeans.centroids.T
classifications = np.argmax(cos_similarities, axis=1)

In [15]:
pd.Series(classifications).value_counts()

1    2378
4    1550
6    1448
2    1019
7     965
8     650
0     555
9     551
3     445
5     439
dtype: int64

In [16]:
pd.Series(kmeans.index.search(dim_reduced_vecs.astype(np.float32), 1)[1].flatten()).value_counts()

1    2378
4    1550
6    1448
2    1019
7     965
8     650
0     555
9     551
3     445
5     439
dtype: int64

In [17]:
prompt_ids = df["prompt_id"]
prompt_ids = prompt_ids.to_numpy()
prompt_ids

array([ 1,  1,  1, ..., 10, 10, 10])

In [18]:
# Get most common centroid for each 1000 points (same label)
max_centroid_per_label = [pd.Series(classifications[i * 1000:(i + 1) * 1000]).value_counts().idxmax() for i in range(10)]
max_centroid_per_label

[1, 4, 0, 1, 9, 4, 4, 1, 1, 7]

In [19]:
# Get most common label for each point classified to a centroid (same centroid)
centroid_labels = [np.where(classifications == i)[0] for i in range(10)]
max_label_per_centroid = [pd.Series(prompt_ids[centroid_labels[i]]).value_counts().idxmax() for i in range(10)]
max_label_per_centroid

[3, 1, 5, 3, 6, 4, 9, 10, 8, 5]

In [20]:
max_centroids = [centroid for centroid in max_centroid_per_label for _ in range(1000)]

# max_labels = [label for label in max_label_per_centroid for _ in range(1000)] #DOESN'T MAKE SENSE - INDICES ARE CENTROIDS, NOT LABELS - DOESN'T MATCH DATA

In [21]:
np.array(max_centroids)

array([1, 1, 1, ..., 7, 7, 7])

In [22]:
label_to_centroid = {idx + 1 : max_centroid_per_label[idx] for idx in range(len(max_centroid_per_label))}

centroid_to_label = {idx : max_label_per_centroid[idx] for idx in range(len(max_label_per_centroid))}

In [23]:
label_to_centroid

{1: 1, 2: 4, 3: 0, 4: 1, 5: 9, 6: 4, 7: 4, 8: 1, 9: 1, 10: 7}

In [24]:
centroid_to_label

{0: 3, 1: 1, 2: 5, 3: 3, 4: 6, 5: 4, 6: 9, 7: 10, 8: 8, 9: 5}

In [25]:
centroid_to_label_2 = {v: k for k, v in label_to_centroid.items()}

In [26]:
for k, v in enumerate(centroid_to_label):
    if(label_to_centroid.get(v) == k):
        centroid_to_label_2[k] = v
    
    if(v not in centroid_to_label_2):
        centroid_to_label_2[k] = centroid_to_label[k]

In [40]:
centroid_to_label_2

{1: 1, 4: 7, 0: 3, 9: 5, 7: 10, 2: 5, 3: 3, 5: 4, 6: 9, 8: 8}

In [27]:
print(pd.Series(prompt_ids[centroid_labels[0]]).value_counts())

print(pd.Series(prompt_ids[centroid_labels[3]]).value_counts())

3    555
dtype: int64
3    445
dtype: int64


In [28]:
print(pd.Series(prompt_ids[centroid_labels[2]]).value_counts())

print(pd.Series(prompt_ids[centroid_labels[9]]).value_counts())

5    435
8    227
1    185
9    166
4      5
2      1
dtype: int64
5    531
8     15
9      5
dtype: int64


In [29]:
vectorized_map = np.vectorize(centroid_to_label.get)
classifications_to_label = vectorized_map(classifications)

classifications_to_label

array([ 9,  9,  9, ..., 10, 10, 10])

In [30]:
vectorized_map = np.vectorize(centroid_to_label_2.get)
classifications_to_label_2 = vectorized_map(classifications)

classifications_to_label_2

array([ 9,  9,  9, ..., 10, 10, 10])

In [31]:
for i in range(10):
    print(f"Prompt {i + 1} Accuracy: ", np.mean(classifications_to_label[i * 1000:(i + 1) * 1000] == (i + 1)))

Prompt 1 Accuracy:  0.371
Prompt 2 Accuracy:  0.0
Prompt 3 Accuracy:  1.0
Prompt 4 Accuracy:  0.172
Prompt 5 Accuracy:  0.966
Prompt 6 Accuracy:  0.413
Prompt 7 Accuracy:  0.0
Prompt 8 Accuracy:  0.267
Prompt 9 Accuracy:  0.229
Prompt 10 Accuracy:  0.965


In [32]:
print(f"Overall Accuracy: ", np.mean(classifications_to_label == prompt_ids))

Overall Accuracy:  0.4383


In [33]:
for i in range(10):
    print(f"Prompt {i + 1} Accuracy: ", np.mean(classifications[i * 1000:(i + 1) * 1000] == max_centroids[i * 1000:(i + 1)*1000]))

Prompt 1 Accuracy:  0.371
Prompt 2 Accuracy:  0.379
Prompt 3 Accuracy:  0.555
Prompt 4 Accuracy:  0.359
Prompt 5 Accuracy:  0.531
Prompt 6 Accuracy:  0.413
Prompt 7 Accuracy:  0.399
Prompt 8 Accuracy:  0.293
Prompt 9 Accuracy:  0.327
Prompt 10 Accuracy:  0.965


In [34]:
print(f"Overall Accuracy: ", np.mean(classifications == max_centroids))

Overall Accuracy:  0.4592


In [35]:
for i in range(10):
    print(f"Prompt {i + 1} Accuracy: ", np.mean(classifications_to_label_2[i * 1000:(i + 1) * 1000] == (i + 1)))

Prompt 1 Accuracy:  0.371
Prompt 2 Accuracy:  0.0
Prompt 3 Accuracy:  1.0
Prompt 4 Accuracy:  0.172
Prompt 5 Accuracy:  0.966
Prompt 6 Accuracy:  0.0
Prompt 7 Accuracy:  0.399
Prompt 8 Accuracy:  0.267
Prompt 9 Accuracy:  0.229
Prompt 10 Accuracy:  0.965


In [36]:
print(f"Overall Accuracy: ", np.mean(classifications_to_label_2 == prompt_ids))

Overall Accuracy:  0.4369


## Measure Conversion Accuracy

In [37]:
label_to_centroid_vectorized_map = np.vectorize(label_to_centroid.get)
centroid_to_label_vectorized_map = np.vectorize(centroid_to_label.get)
prompt_ids_to_centroids = label_to_centroid_vectorized_map(prompt_ids)
max_centroids_to_labels = centroid_to_label_vectorized_map(max_centroids)

In [38]:
np.mean(max_centroids == prompt_ids_to_centroids) #100% - max_centroids is the prompt ids converted to the max centroid corresponding to each label

1.0

In [39]:
np.mean(max_centroids_to_labels == prompt_ids) #<100% - 7/10 label to centroid / centroid to label mappings coincide

0.5